In [2]:
# In[] model load
import pandas as pd
import panel as pn
import dask.dataframe as dd
import numpy as np
from io import BytesIO
import random
import torch
import torch.nn as nn
import torch.optim as optim
from io import StringIO


css = ['https://cdn.datatables.net/1.10.24/css/jquery.dataTables.min.css',
       # Below: Needed for export buttons
       'https://cdn.datatables.net/buttons/1.7.0/css/buttons.dataTables.min.css'
]
js = {
    '$': 'https://code.jquery.com/jquery-3.5.1.js',
    'DataTable': 'https://cdn.datatables.net/1.10.24/js/jquery.dataTables.min.js',
    # Below: Needed for export buttons
    'buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/dataTables.buttons.min.js',
    'jszip': 'https://cdnjs.cloudflare.com/ajax/libs/jszip/3.1.3/jszip.min.js',
    'pdfmake': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/pdfmake.min.js',
    'vfsfonts': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/vfs_fonts.js',
    'html5buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/buttons.html5.min.js',
}
pn.extension(sizing_mode='stretch_width',css_files=css, js_files=js)


## MODEL A

In [3]:
class MLPModel(nn.Module): # 원래조건 : (5,20) (20,15), (15,10), (10,1)
    def __init__(self): 
        super(MLPModel, self).__init__()
        self.linear1 = nn.Linear(6,24)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(24,18)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(18,12)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(12,1)

    def forward(self, x):
    # 인스턴스(샘플) x가 인풋으로 들어왔을 때 모델이 예측하는 y값을 리턴합니다.
        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x
    
model = MLPModel()
model.load_state_dict(torch.load('full-para_training_model.pth'))

<All keys matched successfully>

## DNN A

### One site

In [4]:
text_input = pn.widgets.TextInput(name='pH', placeholder='Enter a value here...',sizing_mode='fixed')
text_input2 = pn.widgets.TextInput(name='Ca (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input3 = pn.widgets.TextInput(name='Mg (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input4 = pn.widgets.TextInput(name='Na (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input5 = pn.widgets.TextInput(name='Alkalinity (㎎ CaCo3/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input6 = pn.widgets.TextInput(name='Dissolved organic carbon (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input7 = pn.widgets.TextInput(name='Dissolved Cu (㎍/L)', placeholder='Enter a value here...',sizing_mode='fixed')
widget_box=pn.Column(pn.pane.Markdown('### Input variables'),pn.Column(pn.Row(text_input, text_input2, text_input3),pn.Row(text_input4, text_input5, text_input6, text_input7)))

In [5]:
def calculate_A():
    if text_input.value and text_input2.value and text_input3.value and text_input4.value and text_input5.value and text_input6.value =='': 
        kr_test_reshape_NN = 'NONE'
    else :
        list=[float(text_input.value),float(text_input4.value),float(text_input3.value),float(text_input2.value),float(text_input6.value),float(text_input5.value)]
        data=np.array(list)
        kr_test_input_x = torch.Tensor(data)
        kr_test_reshape_NN = torch.reshape(model(kr_test_input_x), (-1,))
    return kr_test_reshape_NN

In [6]:
output1 = pn.widgets.TextInput(name='Copper BLM-based PNECs',value='',disabled=True,sizing_mode='fixed')
button = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')
output2 = pn.widgets.TextInput(name='RCR',value='',disabled=True,sizing_mode='fixed')
button2 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

In [7]:
def button_event(event):
    output1.value= str(calculate_A())
button.on_click(button_event)

In [8]:
def button_event2(event):
    x=float(text_input7.value)/calculate_A()
    output2.value= str(x)
button2.on_click(button_event2)

In [9]:
main = pn.Column(widget_box,pn.Column(pn.pane.Markdown('<br>'),pn.Row(pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output1,button)),pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output2,button2)))))

### Batch file

In [10]:
file_input = pn.widgets.FileInput(accept='.csv,.json',sizing_mode='fixed')
button7 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

def csv_input():
    if file_input.value is None:
        stock_file = 'kr_test.csv'
    else : 
        stock_file = BytesIO()
        stock_file.write(file_input.value)
        stock_file.seek(0)
    return stock_file

@pn.depends(button7.param.clicks)
def calculate_A_batch(_):
    if csv_input() == 'kr_test.csv':
        kr_test_data = pd.read_csv(csv_input())
        kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC", "Alkalinity"]] 

        df=kr_test_data

        script = """
        <script>
        if (document.readyState === "complete") {
        $('.example').DataTable();
        } else {
        $(document).ready(function () {
            $('.example').DataTable();
        })
        }
        </script>
        """
        html = df.to_html(classes=['example', 'panel-df'])
        test=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

    else:
        kr_test_data = pd.read_csv(csv_input())
        kr_test_data2=kr_test_data.copy()
        if 'Dissolved Cu' in kr_test_data :
            kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC", "Alkalinity"]] 
            kr_test_X = kr_test_data.iloc[:,:6]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model(kr_test_input_x), (-1,))
            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data,a,left_index=True,right_index=True)
            a['RCR']=a['BLM-based PNECs'] / kr_test_data2['Dissolved Cu']

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))
            
        else:
            kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC", "Alkalinity"]] 

            kr_test_X = kr_test_data.iloc[:,:6]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model(kr_test_input_x), (-1,))
            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data,a,left_index=True,right_index=True)

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

        def file():
            sio=StringIO()
            df.to_csv(sio, encoding="EUC-KR")
            sio.seek(0)
            return sio

        file_download=pn.widgets.FileDownload(callback=file, filename="Calculate_MODEL_data.csv",sizing_mode='fixed')
        test=pn.Column(file_download,table)

    return test

## model B

In [11]:
class MLPModel_B(nn.Module): # 원래조건 : (5,20) (20,15), (15,10), (10,1)
    def __init__(self): 
        super(MLPModel_B, self).__init__()
        self.linear1 = nn.Linear(5,20)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(20,15)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(15,10)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(10,1)

    def forward(self, x):
    # 인스턴스(샘플) x가 인풋으로 들어왔을 때 모델이 예측하는 y값을 리턴합니다.
        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x
    
    
model_B = MLPModel_B()
model_B.load_state_dict(torch.load('no_alk_training_model_v1.pth'))
#model(train_input_x)

<All keys matched successfully>

## DNN B

### One site

In [12]:
text_input8 = pn.widgets.TextInput(name='pH', placeholder='Enter a value here...',sizing_mode='fixed')
text_input9 = pn.widgets.TextInput(name='Ca (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input10 = pn.widgets.TextInput(name='Mg (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input11 = pn.widgets.TextInput(name='Na (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input12 = pn.widgets.TextInput(name='Dissolved organic carbon (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input13 = pn.widgets.TextInput(name='Dissolved Cu (㎍/L)', placeholder='Enter a value here...',sizing_mode='fixed')
widget_box2=pn.Column(pn.pane.Markdown('### Input variables'),pn.Column(pn.Row(text_input8, text_input9, text_input10),pn.Row( text_input11, text_input12, text_input13)))

In [13]:
def calculate_B():
    if text_input8.value and text_input9.value and text_input10.value and text_input11.value and text_input12.value =='': 
        kr_test_reshape_NN = 0
    else :
        list=[float(text_input8.value),float(text_input11.value),float(text_input10.value),float(text_input9.value),float(text_input12.value)]
        data=np.array(list)
        kr_test_input_x = torch.Tensor(data)
        kr_test_reshape_NN = torch.reshape(model_B(kr_test_input_x), (-1,))
    return kr_test_reshape_NN

In [14]:
output3 = pn.widgets.TextInput(name='Copper BLM-based PNECs',value='',disabled=True,sizing_mode='fixed')
button3 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')
output4 = pn.widgets.TextInput(name='RCR',value='',disabled=True,sizing_mode='fixed')
button4 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

In [15]:
def button_event3(event):
    output3.value= str(calculate_B())
button3.on_click(button_event3)

In [16]:
def button_event4(event):
    x=float(text_input13.value)/calculate_B()
    output4.value= str(x)
button4.on_click(button_event4)

In [17]:
main2 = pn.Column(widget_box2,pn.Column(pn.pane.Markdown('<br>'),pn.Row(pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output3,button3)),pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output4,button4)))))

## BATHFILE_B

In [18]:
file_input_B = pn.widgets.FileInput(accept='.csv,.json',sizing_mode='fixed')
button8 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

def csv_input_B():
    if file_input_B.value is None:
        stock_file = 'kr_test.csv'
    else : 
        stock_file = BytesIO()
        stock_file.write(file_input_B.value)
        stock_file.seek(0)
    return stock_file

@pn.depends(button8.param.clicks)
def calculate_B_batch(_):
    if csv_input_B() == 'kr_test.csv':
        kr_test_data = pd.read_csv(csv_input_B())
        kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC"]] 

        df=kr_test_data

        script = """
        <script>
        if (document.readyState === "complete") {
        $('.example').DataTable();
        } else {
        $(document).ready(function () {
            $('.example').DataTable();
        })
        }
        </script>
        """
        html = df.to_html(classes=['example', 'panel-df'])
        test=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

    else:
        kr_test_data = pd.read_csv(csv_input_B())
        kr_test_data2=kr_test_data.copy()
        if 'Dissolved Cu' in kr_test_data :
            kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC"]] 

            kr_test_X = kr_test_data.iloc[:,:5]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model_B(kr_test_input_x), (-1,))
            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data,a,left_index=True,right_index=True)
            a['RCR']=a['BLM-based PNECs'] / kr_test_data2['Dissolved Cu']

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))
        else:
            kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC"]] 

            kr_test_X = kr_test_data.iloc[:,:5]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model_B(kr_test_input_x), (-1,))
            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data,a,left_index=True,right_index=True)

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))
            
        def file():
            sio=StringIO()
            df.to_csv(sio, encoding="EUC-KR")
            sio.seek(0)
            return sio

        file_download=pn.widgets.FileDownload(callback=file, filename="Calculate_MODEL2_data.csv",sizing_mode='fixed')
        test=pn.Column(file_download,table)

    return test

## MODEL C

In [19]:
class MLPModel_C(nn.Module): # 원래조건 : (5,20) (20,15), (15,10), (10,1)
    def __init__(self): 
        super(MLPModel_C, self).__init__()
        self.linear1 = nn.Linear(3,12)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(12,9)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(9,6)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(6,1)

    def forward(self, x):
    # 인스턴스(샘플) x가 인풋으로 들어왔을 때 모델이 예측하는 y값을 리턴합니다.
        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x
    
model_C = MLPModel_C()
model_C.load_state_dict(torch.load('cond_training_model_correct_MLR_v1.pth'))
#model(train_input_x)

<All keys matched successfully>

## DNN C

### One site

In [20]:
text_input14 = pn.widgets.TextInput(name='pH', placeholder='Enter a value here...',sizing_mode='fixed')
text_input15 = pn.widgets.TextInput(name='Electronical conductivity (㎲/㎝)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input16 = pn.widgets.TextInput(name='Dissolved organic carbon (㎎/L)', placeholder='Enter a value here...',sizing_mode='fixed')
text_input17 = pn.widgets.TextInput(name='Dissolved Cu', placeholder='Enter a value here...',sizing_mode='fixed')
widget_box3=pn.Column(pn.pane.Markdown('### Input variables'),pn.Row(text_input14, text_input15, text_input16, text_input17))

In [21]:
def calculate_C():
    if text_input14.value and text_input15.value and text_input16.value =='': 
        kr_test_reshape_NN = 0
    else :
        x=float(text_input15.value)
        y=float(text_input16.value)
        list=[float(text_input14.value),np.log10(x),np.log10(y)]
        data=np.array(list)
        kr_test_input_x = torch.Tensor(data)
        kr_test_reshape_NN = torch.reshape(model_C(kr_test_input_x), (-1,))
    return kr_test_reshape_NN

In [22]:
output5 = pn.widgets.TextInput(name='Copper BLM-based PNECs',value='',disabled=True,sizing_mode='fixed')
button5 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')
output6 = pn.widgets.TextInput(name='RCR',value='',disabled=True,sizing_mode='fixed')
button6 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

In [23]:
def button_event5(event):
    output5.value= str(calculate_C())
button5.on_click(button_event5)

In [24]:
def button_event6(event):
    x=float(text_input14.value)/calculate_C()
    output6.value= str(x)
button6.on_click(button_event6)

In [25]:
main3 = pn.Column(widget_box3,pn.Column(pn.pane.Markdown('<br>'),pn.Row(pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output5,button5)),pn.Column(pn.pane.Markdown('### Output variables'),pn.Row(output6,button6)))))

## BATCH FILE C

In [30]:
file_input_C = pn.widgets.FileInput(accept='.csv,.json',sizing_mode='fixed')
button9 = pn.widgets.Button(name='Calculate', button_type='primary',sizing_mode='fixed')

def csv_input_C():
    if file_input_C.value is None:
        stock_file = 'kr_test.csv'
    else : 
        stock_file = BytesIO()
        stock_file.write(file_input_C.value)
        stock_file.seek(0)
    return stock_file

@pn.depends(button9.param.clicks)
def calculate_C_batch(_):
    if csv_input_C() == 'kr_test.csv':
        kr_test_data = pd.read_csv(csv_input_C())
        kr_test_data = kr_test_data[["pH", "Cond", "DOC",]] 

        df=kr_test_data

        script = """
        <script>
        if (document.readyState === "complete") {
        $('.example').DataTable();
        } else {
        $(document).ready(function () {
            $('.example').DataTable();
        })
        }
        </script>
        """
        html = df.to_html(classes=['example', 'panel-df'])
        test=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

    else:
        kr_test_data = pd.read_csv(csv_input_C())
        kr_test_data2=kr_test_data.copy()

        if 'Dissolved Cu' in kr_test_data :
            kr_test_data = kr_test_data[["pH", "Cond", "DOC",]] 
            kr_test_data3=kr_test_data.copy()
            kr_test_data['Cond'] = np.log10(kr_test_data['Cond'])
            kr_test_data['DOC'] = np.log10(kr_test_data['DOC'])
            kr_test_X = kr_test_data.iloc[:,:3]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model_C(kr_test_input_x), (-1,))

            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data3,a,left_index=True,right_index=True)
            a['RCR']=a['BLM-based PNECs'] / kr_test_data2['Dissolved Cu']

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

        else:
            kr_test_data = kr_test_data[["pH", "Cond", "DOC",]] 
            kr_test_data3=kr_test_data.copy()
            kr_test_data['Cond'] = np.log10(kr_test_data['Cond'])
            kr_test_data['DOC'] = np.log10(kr_test_data['DOC'])
            kr_test_X = kr_test_data.iloc[:,:3]

            kr_test_input_x = torch.Tensor(kr_test_X.values)

            kr_test_reshape_NN = torch.reshape(model_C(kr_test_input_x), (-1,))

            a = kr_test_reshape_NN.detach().numpy()
            a = pd.DataFrame(a)
            a.columns=['BLM-based PNECs']
            a=pd.merge(kr_test_data3,a,left_index=True,right_index=True)

            df=a
            script = """
            <script>
            if (document.readyState === "complete") {
            $('.example').DataTable();
            } else {
            $(document).ready(function () {
                $('.example').DataTable();
            })
            }
            </script>
            """
            html = df.to_html(classes=['example', 'panel-df'])
            table=pn.Column(pn.pane.Markdown("Data frame"),pn.pane.HTML(html+script, sizing_mode='stretch_width'))

        def file():
            sio=StringIO()
            df.to_csv(sio, encoding="EUC-KR")
            sio.seek(0)
            return sio

        file_download=pn.widgets.FileDownload(callback=file, filename="Calculate_MODEL3_data.csv",sizing_mode='fixed')
        test=pn.Column(file_download,table)

    return test

## MARK DOWN

In [52]:
mark=pn.pane.Markdown("#### DOC: dissolved organic carbon ㎎/L, EC: electronical conductivity ㎲/㎝, Unit of Ca, Mg and Na: dissolved concentration ㎎/L, Unit of alkalinity: ㎎ CaCO3/L")
mark2=pn.pane.Markdown("---")
mark3=pn.pane.Markdown("## Batchfile <br> <br> The CSV file format to be input must be the same as the sample format below <br> * If 'Dissolved Cu' exists in your csv column, The RCR is calculated")

In [42]:
mmain1=pn.Column(mark3,pn.Row(file_input,button7),calculate_A_batch,mark)

In [43]:
mmain2=pn.Column(mark3,pn.Row(file_input_B,button8),calculate_B_batch,mark)

In [44]:
mmain3=pn.Column(mark3,pn.Row(file_input_C,button9),calculate_C_batch,mark)

## TAB

In [45]:
dnn_tab = pn.Tabs(
    ("DNN(a)",pn.Column(main,mark2,mmain1)),
    ("DNN(b)",pn.Column(main2,mark2,mmain2)),
    ("DNN(c)",pn.Column(main3,mark2,mmain3)),
    dynamic=True
)

## AREA SELECT

## SERVE

In [46]:
template = pn.template.FastListTemplate(
    site="EHR&C", title="DNN model for BLM vol.1" ,
    main=[dnn_tab]
).servable()

Markdown(str, sizing_mode='stretch_width')